AIM:
To rate and rank the hotels in a particular area based on following parameters:
Number of likes 
Number of trending places nearby 
Existing Rating

TARGET AUDIENCE : Tourists looking to book hotel rooms/stays
APPLICATION:
When a tourist is trying to book a hotel room at a particular tourist destination ,then the hotels will be graded based on the above mentioned parameters and thus giving the end user the best possible experience.

DATAFLOW :
1. Get the list of hotels in a NY
2. For each hotel ,calculate a grade point using the above criteria
3. Filter the hotels if they are not upto the mark.
4. Sort the hotels based on the grade and recommend to the tourist
5. For the most recommended hotel, use Folium to plot all the nearby trending venues 

Importing all necessary libraries

In [0]:
import pandas as pd
import numpy as np
import folium  
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib
from IPython.core.display import HTML 
import requests

Get the co-ordinates of Agra using geocoding

In [0]:
initialize =Nominatim(user_agent=" Discover places")
location=initialize.geocode("New York")
lat=location.latitude
longi=location.longitude

Initializing the REST API call parameters

In [0]:
Client_id="ILLKE4HIVXHZMSXICFFHHNLXQ3P4AXL2UK12NIGKJL00NJZG"
Client_Secret="PZE32RDY1IEBREUSDVOEWX3BTW1ZSTF535YW3TVF4QDW4Z22"
version="20180604"
radius=1000
query="Hotel"


Using Explore Venue to get list of all places in the given latitude and longitude

In [0]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}'.format(
            Client_id, 
            Client_Secret, 
            version, 
            lat, 
            longi, 
            radius, 
            query,
            
            )
    
    
results = requests.get(url).json()
venue_data=results["response"]['groups'][0]['items']
venue_details=[]


Extracting the venue id, name and category for each venue

In [5]:
for row in venue_data:
    try:
        venue_id=row['venue']['id']
        venue_name=row['venue']['name']
        venue_category=row['venue']['categories'][0]['name']
        venue_details.append([venue_id,venue_name,venue_category])
    except KeyError:
         pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names) 

df 



,ID,Name,Category
0,56d8c0f8498edb854f926e6a,"The Beekman, A Thompson Hotel",Hotel
1,57c640ad498e74977f98372f,Four Seasons Hotel New York Downtown,Hotel
2,49efcc88f964a52006691fe3,Smyth Hotel,Hotel
3,54aed577498ee5fc196739f7,Residence Inn by Marriott New York Downtown Ma...,Hotel
4,5bd4cc646adbf5002c1ced5c,Moxy NYC Downtown,Hotel
5,3fd66200f964a520bbe61ee3,The Roxy Hotel,Hotel
6,4ab6c985f964a5200a7920e3,Greenwich Hotel,Hotel
7,4a6e3e75f964a5204ed41fe3,"Gild Hall, A Thompson Hotel",Hotel
8,547dc381498ec37f84f01439,Aloft Manhattan Downtown - Financial District,Hotel
9,4f22ca77e4b0ed3396a83a05,Conrad,Hotel


For each venue we get the number of likes, ratings and the co-ordinates

In [6]:
df1=pd.DataFrame()
for i in df.index:
    venue_id=df['ID'][i]
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            Client_id, 
            Client_Secret, 
            version)
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]

    venue_id=venue_data['id']
    venue_name=venue_data['name']
    venue_likes=venue_data['likes']['count']
    venue_rating=venue_data['rating']
    venue_lat=venue_data['location']['lat']
    venue_long=venue_data['location']['lng']
    venue_details.extend([venue_id,venue_name,venue_lat,venue_long,venue_likes,venue_rating])
   
     
    data={"ID":venue_id,"Name":venue_name,"Latitude":venue_lat,"Longitude":venue_long,"Likes":venue_likes,"Ratings":venue_rating}
    df1=df1.append(data,ignore_index=True)
    
df1

,ID,Latitude,Likes,Longitude,Name,Ratings
0,56d8c0f8498edb854f926e6a,40.711173,208.0,-74.006702,"The Beekman, A Thompson Hotel",9.2
1,57c640ad498e74977f98372f,40.712612,67.0,-74.009380,Four Seasons Hotel New York Downtown,9.2
2,49efcc88f964a52006691fe3,40.715144,62.0,-74.009183,Smyth Hotel,8.4
3,54aed577498ee5fc196739f7,40.709563,54.0,-74.009718,Residence Inn by Marriott New York Downtown Ma...,8.1
4,5bd4cc646adbf5002c1ced5c,40.710662,31.0,-74.007969,Moxy NYC Downtown,7.8
5,3fd66200f964a520bbe61ee3,40.719341,496.0,-74.005044,The Roxy Hotel,8.7
6,4ab6c985f964a5200a7920e3,40.719876,99.0,-74.009949,Greenwich Hotel,8.9
7,4a6e3e75f964a5204ed41fe3,40.707887,66.0,-74.007103,"Gild Hall, A Thompson Hotel",8.0
8,547dc381498ec37f84f01439,40.710394,67.0,-74.006704,Aloft Manhattan Downtown - Financial District,7.3
9,4f22ca77e4b0ed3396a83a05,40.715035,293.0,-74.015840,Conrad,8.7


Getting the total number of trending places for each hotel and calculating the total score earned

In [7]:
df1['Trending']= ""
df1['Score']=""
#Introducing a new empty column
for i in df1.index:
    lat=df1['Latitude'][i]
    lng=df1['Longitude'][i]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}'.format(Client_id, 
           Client_Secret, 
           lat,lng,version,radius
           )
    
    results = requests.get(url).json()
    details=results['response']['totalResults']
    df1['Trending'][i]= details
    df1['Score'][i] = df1['Likes'][i] + df1['Ratings'][i]*3 + df1['Trending'][i]*3

column_names=["ID","Name","Latitude","Longitude","Likes","Ratings","Trending","Score"]
df1=df1[column_names]

#Sorting the hotels according to their scores obtained.Re-arranging the columns

df1.sort_values("Score", axis = 0, ascending = False, 
                 inplace = True) 
df1.reset_index(inplace=True)
column_names=['ID','Name','Latitude','Longitude',	'Likes','Ratings','Trending','Score']
df1=df1[column_names]

df1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,ID,Name,Latitude,Longitude,Likes,Ratings,Trending,Score
0,3fd66200f964a520bbe61ee3,The Roxy Hotel,40.719341,-74.005044,496.0,8.7,231,1215.1
1,5547ffe8498ec8a2c668a412,NOMO SOHO,40.719892,-74.000083,328.0,8.3,235,1057.9
2,4f22ca77e4b0ed3396a83a05,Conrad,40.715035,-74.015840,293.0,8.7,227,1000.1
3,4bc7ab4c14d79521c6e367e9,W New York - Downtown,40.709137,-74.013651,249.0,7.8,231,965.4
4,56d8c0f8498edb854f926e6a,"The Beekman, A Thompson Hotel",40.711173,-74.006702,208.0,9.2,238,949.6
5,57f0689d498e7d49d9189369,The Bar Room at Temple Court,40.711448,-74.006802,192.0,9.4,238,934.2
6,4b1b13ddf964a520bbf723e3,Andaz Wall Street - A Concept By Hyatt,40.705100,-74.007868,183.0,8.1,225,882.3
7,510ad6e2e4b0a2ecec03b322,Millennium Hilton,40.711114,-74.010333,142.0,7.4,235,869.2
8,4a82eea3f964a52090f91fe3,New York Marriott Downtown,40.709504,-74.014672,149.0,7.7,232,868.1
9,56f1dac5498ee078791a2051,11 Howard,40.719153,-74.000221,116.0,8.5,234,843.5


For the selected hotel , we get the list of all the trending places

In [8]:
from IPython.display import Image 
from IPython.core.display import HTML 

df2=pd.DataFrame(df1.iloc[0])
lat=float(df2.loc['Latitude'])
lng=float(df2.loc['Longitude'])
name=str(df2.loc['Name'])

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}'.format(Client_id, 
           Client_Secret, 
           lat,lng,version,radius
           )
    
results = requests.get(url).json()
trending_venues = results['response']['groups'][0]['items']
trending_venues_df = pd.json_normalize(trending_venues)

# filter columns
filtered_df = pd.DataFrame(trending_venues_df[['venue.name','venue.location.lat','venue.location.lng']])
filtered_df



,venue.name,venue.location.lat,venue.location.lng
0,Aire Ancient Baths,40.718141,-74.004941
1,SKY TING YOGA,40.718124,-74.003051
2,Gotan,40.719217,-74.006642
3,Spring Studios,40.720915,-74.006207
4,The Django,40.719365,-74.004880
5,Terra Wine Bar,40.719268,-74.006505
6,Macao Trading Co.,40.719759,-74.003981
7,Brandy Library,40.719952,-74.007412
8,Best Made Company,40.718881,-74.004745
9,La Colombe Torrefaction,40.719829,-74.003854


Plotting the map of NY and the hotel selected

In [16]:

#Map of New York City
m = folium.Map(
    location=[ 40.7187,-74.0010],
    zoom_start=20,
)


#Plotting the selected hotel in Red
lat=float(df2.loc['Latitude'])
lng=float(df2.loc['Longitude'])
folium.CircleMarker(
    location=[lat,lng],
    radius=10,
    popup=name,
    fill=True,
    color='red',
    fill_color='red', 
    fill_opacity=0.6
).add_to(m)

m


ERROR! Session/line number was not unique in database. History logging moved to new session 59


Plotting the list of all the trending places for the given hotel

In [17]:

for lat, lng, name in zip(filtered_df['venue.location.lat'], filtered_df['venue.location.lng'], 
                                           filtered_df['venue.name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3,
        parse_html=False).add_to(m)  
    
m